# TreeDex Demo — Gemini + PDF

**Tree-based, vectorless RAG** — index any document, query with any LLM.

This notebook:
1. Installs TreeDex from GitHub
2. Connects to **Google Gemini** (free API)
3. Uploads a PDF and builds a tree index
4. Queries the index

---

## 1. Install TreeDex

In [ ]:
!pip install -q google-generativeai git+https://github.com/mithun50/TreeDex.git

In [ ]:
import treedex
print(f"TreeDex v{treedex.__version__} installed!")

## 2. Setup Gemini LLM

1. Get a free API key at: https://aistudio.google.com/apikey
2. In Colab sidebar, click the **key icon** (Secrets)
3. Add a secret named `GEMINI_API_KEY` with your key
4. **Toggle "Notebook access" ON** for that secret

In [ ]:
from treedex import GeminiLLM

try:
    from google.colab import userdata
    GEMINI_KEY = userdata.get("GEMINI_API_KEY")
    print(f"Key loaded from Colab secrets ({len(GEMINI_KEY)} chars)")
except Exception as e:
    print(f"Could not load from secrets: {e}")
    GEMINI_KEY = input("Paste your Gemini API key: ").strip()

llm = GeminiLLM(api_key=GEMINI_KEY)
print(f"Using: {llm}")

In [ ]:
# Test the API key
try:
    response = llm.generate("Say 'hello' in one word.")
    print(f"API test passed! Response: {response}")
except Exception as e:
    print(f"API test FAILED: {e}")
    print("\nGet a free key at: https://aistudio.google.com/apikey")

## 3. Upload a PDF

In [ ]:
from google.colab import files as colab_files

uploaded = colab_files.upload()
pdf_name = list(uploaded.keys())[0]
print(f"Uploaded: {pdf_name}")

## 4. Build the Index

In [ ]:
from treedex import TreeDex

index = TreeDex.from_file(pdf_name, llm=llm)

## 5. Inspect

In [ ]:
index.show_tree()

In [ ]:
stats = index.stats()
for k, v in stats.items():
    print(f"  {k}: {v}")

## 6. Save & Load

In [ ]:
index.save("my_index.json")
print("Saved!")

index2 = TreeDex.load("my_index.json", llm=llm)
print(f"Loaded: {index2.stats()['total_nodes']} nodes")

## 7. Query

In [ ]:
result = index.query("What are the main topics covered in this document?")

print(f"Nodes: {result.node_ids}")
print(f"Source: {result.pages_str}")
print(f"Reasoning: {result.reasoning}")
print(f"\nContext ({len(result.context)} chars):")
print(result.context[:1000])

In [ ]:
result2 = index.query("Explain the key concepts in the first chapter.")

print(f"Nodes: {result2.node_ids}")
print(f"Source: {result2.pages_str}")
print(f"Reasoning: {result2.reasoning}")

In [ ]:
# Try your own question
q = input("Your question: ")
r = index.query(q)
print(f"\nSource: {r.pages_str}")
print(f"Reasoning: {r.reasoning}")
print(f"\n{r.context[:1500]}")